In [39]:
import numpy as np
import pandas as pd
import sys
import random

# Distance matrix

In [40]:
#an improved version of the read_subst_mtrx from project_3
#returns the matrix and a dictionary mapping the species name to indeces
def read_mtrx(filename):
    f = open(filename,'r')
    num_species = int(f.readline())

    dict_mat = {}
    mat = np.zeros((num_species,num_species))
    
    for i in range(0,num_species):
        line = f.readline()
        nums_in_line = line.split()
        dict_mat[nums_in_line[0]] = i
        for j in range(1,num_species +1):
            mat[i,j-1] = nums_in_line[j]
    f.close()
    return dict_mat, mat 

In [232]:
species_dict, dist = read_mtrx("C:/Users/lenab/Documents/AU/Algorithms_in_bioinformatics/Brouillons_projets/example_slide4.phy")
print(species_dict)
print(dist)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
[[0.   0.23 0.16 0.2  0.17]
 [0.23 0.   0.23 0.17 0.24]
 [0.16 0.23 0.   0.2  0.11]
 [0.2  0.17 0.2  0.   0.21]
 [0.17 0.24 0.11 0.21 0.  ]]


In [233]:
species = list(species_dict.keys())
dict_dist = {}
for i in range(len(species)) :
    dict_dist[species[i]] = dist[i]
dict_dist

{'A': array([0.  , 0.23, 0.16, 0.2 , 0.17]),
 'B': array([0.23, 0.  , 0.23, 0.17, 0.24]),
 'C': array([0.16, 0.23, 0.  , 0.2 , 0.11]),
 'D': array([0.2 , 0.17, 0.2 , 0.  , 0.21]),
 'E': array([0.17, 0.24, 0.11, 0.21, 0.  ])}

In [234]:
dist=pd.DataFrame.from_dict(dict_dist,
orient='index', columns=species)
print(dist)

      A     B     C     D     E
A  0.00  0.23  0.16  0.20  0.17
B  0.23  0.00  0.23  0.17  0.24
C  0.16  0.23  0.00  0.20  0.11
D  0.20  0.17  0.20  0.00  0.21
E  0.17  0.24  0.11  0.21  0.00


In [44]:
dist.loc['A','B']

0.23

# NJ algorithm

## Step 1

In [45]:
N = pd.DataFrame(columns=species, index = species)
N

,A,B,C,D,E
A,NaN,NaN,NaN,NaN,NaN
B,NaN,NaN,NaN,NaN,NaN
C,NaN,NaN,NaN,NaN,NaN
D,NaN,NaN,NaN,NaN,NaN
E,NaN,NaN,NaN,NaN,NaN


In [46]:
dist.loc['A','B']

0.23

In [235]:
def step1(dist):
    S = len(dist.columns)
    N = pd.DataFrame(columns=dist.columns, index = dist.index)
    rs = []
    for i in dist.index:
        ri = 0
        for j in dist.columns:
            ri +=dist.loc[i,j]
        ris.append(ri/(S-2))
        
    minN = sys.maxsize
    mini = dist.index[0]
    minj = dist.columns[1]
    rimin = rs[0]
    rjmin = rs[1]
    index_mini = 0
    index_minj = 1
    for i in range(len(dist.index)):
        for j in range(len(dist.columns)):
            ci = dist.index[i]
            cj = dist.columns[j]
            nij = dist.loc[ci,cj]-rs[i]-rs[j]
            N.loc[ci,cj]=nij
            if ci != cj:
                if nij<minN : 
                    minN = nij
                    mini = ci
                    minj = cj
                    rimin = rs[i]
                    rjmin = rs[j]
                    index_mini = i
                    index_minj = j
                    
    return(index_mini, index_minj, mini,minj,minN,rimin,rjmin)


In [236]:
index_mini, index_minj, mini,minj, minN,rimin, rjmin = step1(dist)
print (index_mini, index_minj, mini,minj, minN,rimin, rjmin )

[0.25333333333333335, 0.29, 0.23333333333333336, 0.26, 0.24333333333333332] [0.25333333333333335, 0.29, 0.23333333333333336, 0.26, 0.24333333333333332]
1 3 B D -0.38 0.29 0.26


## Step 2

In [49]:
nodes = list(dist.index).copy()
print(nodes)

['A', 'B', 'C', 'D', 'E']


In [50]:
def step2(mini,minj,dist):
    nodes = list(dist.index).copy()
    nodes.remove(mini)
    nodes.remove(minj)
    nodes.append('('+mini+','+minj+')')
    return nodes

In [51]:
nodes = step2('B','D',dist)
nodes

['A', 'C', 'E', '(B,D)']

## Step 3

In [52]:
# This function works for the first iteration, but not for the next ones, because we do not keep in memory previous lengths
def step3(mini, minj, dist, rimin, rjmin):
    nodes = list(dist.index).copy()
    nodes.remove(mini)
    nodes.remove(minj)
    #dik = (dist.loc[mini,minj] + rimin - rjmin)/2
    #djk = (dist.loc[mini,minj] + rjmin - rimin)/2
    dik = round((dist.loc[mini,minj] + rimin - rjmin)/2,4)
    djk = round((dist.loc[mini,minj] + rjmin - rimin)/2,4)
    nodes.append('(' + mini + ': '+ str(dik) + ','+ minj + ": "+ str(djk) + ')')
    return nodes

In [53]:
nodes_length = step3(mini, minj, dist, rimin, rjmin)
nodes_length

['A', 'C', 'E', '(B: 0.1,D: 0.07)']

Try to find a solution

In [54]:
nodes_length = list(dist.index).copy()
def step3(index_mini, index_minj, mini, minj, dist, rimin, rjmin, node_length):
    nodes = node_length.copy()
    nodes.remove(node_length[index_mini])
    nodes.remove(node_length[index_minj])
    #dik = (dist.loc[mini,minj] + rimin - rjmin)/2
    #djk = (dist.loc[mini,minj] + rjmin - rimin)/2
    dik = round((dist.loc[mini,minj] + rimin - rjmin)/2,4)
    djk = round((dist.loc[mini,minj] + rjmin - rimin)/2,4)
    nodes.append('(' + node_length[index_mini] + ': '+ str(dik) + ','+ node_length[index_minj]+ ": "+ str(djk) + ')')
    return nodes

In [55]:
step3(index_mini, index_minj, mini, minj, dist, rimin, rjmin, nodes_length)

['A', 'C', 'E', '(B: 0.1,D: 0.07)']

This second function works. => Take this one

## Step 4  

In [56]:
nodes[-1]

'(B,D)'

In [57]:
dist.loc["A",:]

A    0.00
B    0.23
C    0.16
D    0.20
E    0.17
Name: A, dtype: float64

In [58]:
dist.loc["A","B"]=460
dist

,A,B,C,D,E
A,0.00,460.00,0.16,0.20,0.17
B,0.23,0.00,0.23,0.17,0.24
C,0.16,0.23,0.00,0.20,0.11
D,0.20,0.17,0.20,0.00,0.21
E,0.17,0.24,0.11,0.21,0.00


In [59]:
col_j = dist["A"]
print(col_j)
col_j["B"]

A    0.00
B    0.23
C    0.16
D    0.20
E    0.17
Name: A, dtype: float64


0.23

In [60]:
row_i = dist.loc["A",:]
print(row_i)
row_i["E"]

A      0.00
B    460.00
C      0.16
D      0.20
E      0.17
Name: A, dtype: float64


0.17

In [61]:
# Reinitialize dist
dist=pd.DataFrame.from_dict(dict_dist,
orient='index', columns=species)
print(dist)

      A     B     C     D     E
A  0.00  0.23  0.16  0.20  0.17
B  0.23  0.00  0.23  0.17  0.24
C  0.16  0.23  0.00  0.20  0.11
D  0.20  0.17  0.20  0.00  0.21
E  0.17  0.24  0.11  0.21  0.00


In [62]:
k = nodes[-1]
row_mini = dist.loc[mini,:]
col_minj = dist[minj]
for i in dist.index : 
    if i != mini and i!=minj : 
        dist.loc[i,k] = (dist.loc[mini,i]+dist.loc[minj,i]-dist.loc[mini,minj])/2
        dist.loc[k,i] = (dist.loc[mini,i]+dist.loc[minj,i]-dist.loc[mini,minj])/2
dist.loc[k,k]=0
dist = dist.drop(index = [mini,minj], columns = [mini,minj])
dist

,A,C,E,"(B,D)"
A,0.00,0.16,0.17,0.13
C,0.16,0.00,0.11,0.13
E,0.17,0.11,0.00,0.14
"(B,D)",0.13,0.13,0.14,0.00


In [63]:
def step4(mini, minj, dist):
    k = nodes[-1]
    row_mini = dist.loc[mini,:]
    col_minj = dist[minj]
    for i in dist.index : 
        if i != mini and i!=minj : 
            dist.loc[i,k] = (dist.loc[mini,i]+dist.loc[minj,i]-dist.loc[mini,minj])/2
            dist.loc[k,i] = (dist.loc[mini,i]+dist.loc[minj,i]-dist.loc[mini,minj])/2
    dist.loc[k,k]=0
    dist = dist.drop(index = [mini,minj], columns = [mini,minj])
    return(dist)

## Try a second step

In [167]:
# Reinitialize dist
dist=pd.DataFrame.from_dict(dict_dist,
orient='index', columns=species)
### Reinitialize node_length
nodes_length = list(dist.index).copy()
print(dist)
print(nodes_length)

      A     B     C     D     E
A  0.00  0.23  0.16  0.20  0.17
B  0.23  0.00  0.23  0.17  0.24
C  0.16  0.23  0.00  0.20  0.11
D  0.20  0.17  0.20  0.00  0.21
E  0.17  0.24  0.11  0.21  0.00
['A', 'B', 'C', 'D', 'E']


In [173]:
index_mini, index_minj,mini,minj, minN,rimin, rjmin = step1(dist)
print (index_mini, index_minj,mini,minj, minN,rimin, rjmin)

0 3 A (B,D) -0.3 0.23 0.19999999999999998


In [174]:
nodes = step2(mini,minj,dist)
nodes

['C', 'E', '(A,(B,D))']

In [175]:
nodes_length

['A', 'C', 'E', '(B: 0.1,D: 0.07)']

In [176]:
nodes_length = step3(index_mini, index_minj, mini, minj, dist, rimin, rjmin, nodes_length)
nodes_length

['C', 'E', '(A: 0.08,(B: 0.1,D: 0.07): 0.05)']

In [177]:
dist = step4(mini, minj, dist)
dist

,C,E,"(A,(B,D))"
C,0.00,0.11,0.08
E,0.11,0.00,0.09
"(A,(B,D))",0.08,0.09,0.00


In [150]:
len(dist.index)

3

## Termination step

In [151]:
i = nodes[0]
j = nodes[1]
m = nodes[2]
print(i,j,m)
#dvi = (dist.loc[i,j]+dist.loc[i,m]-dist.loc[j,m])/2
#dvj = (dist.loc[i,j]+dist.loc[j,m]-dist.loc[i,m])/2
#dvm = (dist.loc[i,m]+dist.loc[j,m]-dist.loc[i,j])/2
dvi = round((dist.loc[i,j]+dist.loc[i,m]-dist.loc[j,m])/2,3)
dvj = round((dist.loc[i,j]+dist.loc[j,m]-dist.loc[i,m])/2,3)
dvm = round((dist.loc[i,m]+dist.loc[j,m]-dist.loc[i,j])/2,3)
tree = "("+i+":"+str(dvi)+", "+j+":"+str(dvj)+", "+m+":"+str(dvm)+")"
print(tree)

C E (A,(B,D))
(C:0.05, E:0.06, (A,(B,D)):0.03)


## Full NJ

In [230]:
def NJ(dist_mat):
    # create the distance dataframe
    species_dict, dist = read_mtrx(dist_mat)
    species = list(species_dict.keys())
    dict_dist = {}
    for i in range(len(species)) :
        dict_dist[species[i]] = dist[i]
    dist=pd.DataFrame.from_dict(dict_dist, orient='index', columns=species) 
    
    # initializing variables
    nodes_length = list(dist.index).copy()
    nodes = list(dist.index).copy()
    
    while len(dist.index)>3 : 
        
        #step 1 
        index_mini, index_minj,mini,minj, minN,rimin, rjmin = step1(dist)
        
        # step 2 
        nodes = list(dist.index).copy()
        nodes.remove(mini)
        nodes.remove(minj)
        nodes.append('('+mini+','+minj+')')
        
        #step 3 
        nodes2 = nodes_length.copy()
        nodes_length.remove(nodes2[index_mini])
        nodes_length.remove(nodes2[index_minj])
        dik = round((dist.loc[mini,minj] + rimin - rjmin)/2,4)
        djk = round((dist.loc[mini,minj] + rjmin - rimin)/2,4)
        nodes_length.append('(' + nodes2[index_mini] + ': '+ str(dik) + ','+ nodes2[index_minj]+ ": "+ str(djk) + ')')
        
        # step 4
        k = nodes[-1]
        row_mini = dist.loc[mini,:]
        col_minj = dist[minj]
        for i in dist.index : 
            if i != mini and i!=minj : 
                dist.loc[i,k] = (dist.loc[mini,i]+dist.loc[minj,i]-dist.loc[mini,minj])/2
                dist.loc[k,i] = (dist.loc[mini,i]+dist.loc[minj,i]-dist.loc[mini,minj])/2
        dist.loc[k,k]=0
        dist = dist.drop(index = [mini,minj], columns = [mini,minj])
        
    # Termination step
    i = nodes[0]
    j = nodes[1]
    m = nodes[2]
    dvi = (dist.loc[i,j]+dist.loc[i,m]-dist.loc[j,m])/2
    dvj = (dist.loc[i,j]+dist.loc[j,m]-dist.loc[i,m])/2
    dvm = (dist.loc[i,m]+dist.loc[j,m]-dist.loc[i,j])/2
    tree = "("+nodes_length[0]+":"+str(dvi)+", "+nodes_length[1]+":"+str(dvj)+", "+nodes_length[2]+":"+str(dvm)+")"
    return (tree)

In [228]:
NJ("C:/Users/lenab/Documents/AU/Algorithms_in_bioinformatics/Brouillons_projets/example_slide4.phy")

'(C:0.05000000000000002, E:0.05999999999999997, (A: 0.08,(B: 0.1,D: 0.07): 0.05):0.029999999999999992)'

In [229]:
NJ("C:/Users/lenab/Documents/AU/Algorithms_in_bioinformatics/Brouillons_projets/Project5/expe_matrix/89_Adeno_E3_CR1.phy")

'((39_E321_A: 0.0,40_Q5EY51: 0.0):0.0, (41_Q77SL0: 0.0,42_Q77SL8: 0.0):0.0, (43_Q9PX39: 0.0,(38_Q6RK78: 0.0118,(((20_Q6QP93: 0.3281,((63_Q6QP95: 0.0,64_Q8UY72: 0.0): 0.3044,(((53_Q76EL6: 0.011,((54_Q7TBH0: 0.0,55_Q91CB8: 0.0): 0.0108,(51_Q91CB6: 0.0088,52_Q8JZH6: 0.0051): 0.003): 0.0029): 0.2467,((47_Q8JZH7: 0.0312,(45_Q76C87: 0.0276,(44_Q91CB7: 0.0289,46_Q80IV8: 0.0275): 0.0146): 0.0111): 0.1135,(50_Q4KSJ0: 0.1112,(48_Q67812: 0.0,49_Q77N38: 0.0): 0.1): 0.0268): 0.0709): 0.0396,((((0_Q91PL0_: 0.0293,(1_E320_AD: 0.0083,((6_Q76Y40_: 0.0,7_Q781A8_: 0.0): 0.0,((2_E320_AD: 0.0,3_E320_AD: 0.0): 0.0,(4_Q32UG3_: 0.0,5_Q5UVZ8_: 0.0): 0.0): 0.0): 0.0046): 0.0091): 0.0633,(((8_Q6RK79_: 0.0,9_Q77SL1_: 0.0): 0.0,(10_Q9PX68: 0.0,11_Q9QL93: 0.0): 0.0): 0.0096,((12_Q3ZKT4: 0.0,13_Q91CL3: 0.0): 0.007,((18_Q83127: 0.0,19_Q91CK4: 0.0): 0.0,((14_E320_A: 0.0,15_Q2KSH6: 0.0): 0.0,(16_Q2Y0G4: 0.0,17_Q83118: 0.0): 0.0): 0.0): 0.0058): 0.0032): 0.0713): 0.2194,((((77_Q91CB8: 0.0068,78_Q7TBH0: 0.0062): 0.0367,(